In [1]:
using NeuralPDE, Lux, Optimization, OptimizationOptimJL, LineSearches, Plots, OptimizationOptimisers, Statistics, WriteVTK
using ModelingToolkit: Interval

In [2]:
@parameters x y
@variables u(..)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

Differential(y) ∘ Differential(y)

In [3]:
# 2D PDE
eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ 4*(x^2 + y^2 - 1)*exp(-x^2 - y^2)

Differential(y)(Differential(y)(u(x, y))) + Differential(x)(Differential(x)(u(x, y))) ~ 4(-1 + x^2 + y^2)*exp(-(x^2) - (y^2))

In [4]:
# Boundary conditions
bcs = [u(x, -4.0) ~ exp(-x^2 - 16.0),
       u(x, 4.0) ~ exp(-x^2 - 16.0),
       u(-4.0, y) ~ exp(-16.0 - y^2),
       u(4.0, y) ~ exp(-16.0 - y^2)]

4-element Vector{Equation}:
 u(x, -4.0) ~ exp(-16.0 - (x^2))
 u(x, 4.0) ~ exp(-16.0 - (x^2))
 u(-4.0, y) ~ exp(-16.0 - (y^2))
 u(4.0, y) ~ exp(-16.0 - (y^2))

In [5]:
# Space and time domains
domains = [x ∈ Interval(-4.0, 4.0), y ∈ Interval(-4.0, 4.0)]



2-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(x, -4.0 .. 4.0)
 Symbolics.VarDomainPairing(y, -4.0 .. 4.0)

In [6]:
# Neural network
dim = 2 # number of dimensions
chain = Chain(Dense(dim, 32, σ), Dense(32, 32, σ),Dense(32, 32, σ), Dense(32, 1))


Chain(
    layer_1 = Dense(2 => 32, σ),        # 96 parameters
    layer_2 = Dense(32 => 32, σ),       # 1_056 parameters
    layer_3 = Dense(32 => 32, σ),       # 1_056 parameters
    layer_4 = Dense(32 => 1),           # 33 parameters
)         # Total: 2_241 parameters,
          #        plus 0 states.

In [7]:
# Discretization
discretization = PhysicsInformedNN(
    chain, GridTraining(0.1))

PhysicsInformedNN{Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, GridTraining{Float64}, Nothing, Nothing, NeuralPDE.Phi{StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}}, typeof(NeuralPDE.numeric_derivative), Bool, Nothing, Nothing, Nothing, Base.RefValue{Int64}, Base.Pairs{Symbol, Union{}, Tuple{},

In [8]:
@named pde_system = PDESystem(eq, bcs, domains, [x, y], [u(x, y)])
prob = discretize(pde_system, discretization)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(layer_1 = (weight = [-0.1373452991247177 0.541355550289154; -1.015578031539917 -0.0698859915137291; … ; 0.25770968198776245 -0.8901108503341675; -0.9828454256057739 -0.7188675999641418], bias = [-0.16759324073791504, -0.08669109642505646, -0.40589454770088196, -0.17012372612953186, 0.44440776109695435, 0.18005065619945526, -0.09027492254972458, 0.3221401572227478, 0.24671196937561035, 0.47722959518432617  …  0.07176520675420761, 0.41500458121299744, 0.3847815990447998, 0.15770886838436127, 0.31647536158561707, -0.5744693875312805, -0.3053697645664215, 0.6812012791633606, -0.23323695361614227, 0.3764350712299347]), layer_2 = (weight = [-0.0700049102306366 -0.3045763075351715 … -0.14064781367778778 -0.28950613737106323; -0.05398586764931679 -0.17431515455245972 … 0.06083550304174423 -0.3018510341644287; … ; -0.2026083916425705 -0.22047854959964752 … 0.20571967959403992 -0.15440861880779266; 0.17763011157512665 0.09489603340

In [9]:
# plateau‐detection parameters
const PLATEAU_WINDOW = 100    # look at last 100 callback calls
const PLATEAU_TOL    = 1e-5   # max allowed range in that window

1.0e-5

In [10]:
function plateau_callback(p, loss)
    # record
    push!(loss_history, loss)
    push!(time_history, time() - start_time)

    # print progress
    @info "iter=$(length(loss_history))  loss=$(loss)  t=$(time_history[end])"

    # check plateau
    if length(loss_history) ≥ PLATEAU_WINDOW
        recent = @view loss_history[end-PLATEAU_WINDOW+1:end]
        if maximum(recent) - minimum(recent) < PLATEAU_TOL
            @info "⏹️  Early stopping: loss range < $PLATEAU_TOL over last $PLATEAU_WINDOW iters"
            return true   # signal to stop
        end
    end

    return false      # otherwise, keep going
end

plateau_callback (generic function with 1 method)

In [11]:
global loss_history = Float64[]
global time_history = Float64[]
global start_time = time()



# Optimizer
res = solve(prob, OptimizationOptimisers.Adam(0.01), maxiters = 10000, callback=plateau_callback)
phi = discretization.phi

[ Info: iter=1  loss=0.2089527589490128  t=43.67700004577637
[ Info: iter=2  loss=0.6346509709288236  t=44.87299990653992
[ Info: iter=3  loss=0.22761776416682616  t=45.448999881744385
[ Info: iter=4  loss=0.27720457354754335  t=45.520999908447266
[ Info: iter=5  loss=0.4188942403638273  t=45.5939998626709
[ Info: iter=6  loss=0.3433282440544072  t=45.68599987030029
[ Info: iter=7  loss=0.22091664655339383  t=45.759000062942505
[ Info: iter=8  loss=0.19670222747090585  t=46.28699994087219
[ Info: iter=9  loss=0.2578869486544722  t=46.365999937057495
[ Info: iter=10  loss=0.30047148619373476  t=46.42700004577637
[ Info: iter=11  loss=0.2760520640006161  t=46.51600003242493
[ Info: iter=12  loss=0.2217764330400905  t=46.59699988365173
[ Info: iter=13  loss=0.19193426149754908  t=47.11699986457825
[ Info: iter=14  loss=0.20451398487319897  t=47.18499994277954
[ Info: iter=15  loss=0.2344728852721782  t=47.25600004196167
[ Info: iter=16  loss=0.2468354451939606  t=47.31599998474121
[ Info:

NeuralPDE.Phi{StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}}(StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}(Chain{@NamedTuple{layer_1::Dense{typeof(σ

In [14]:
dx = 0.05
xs, ys = [infimum(d.domain):(dx / 10):supremum(d.domain) for d in domains]
analytic_sol_func(x, y) = exp(-x^2 - y^2)

u_predict = reshape([first(phi([x, y], res.u)) for x in xs for y in ys],
    (length(xs), length(ys)))
u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
    (length(xs), length(ys)))
diff_u = abs.(u_predict .- u_real)

p1 = plot(xs, ys, u_real, linetype = :contourf, title = "analytic");
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "predict");
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)

plot(time_history, loss_history,
    xlabel="Time (seconds)", ylabel="Loss",
    title="Loss on time", legend=false)

dx = xs[2] - xs[1]
dy = ys[2] - ys[1]

# Errors
E = u_predict .- u_real

# RMSE (per‐point average)
rmse = sqrt(mean(E.^2))

# Discrete L2 norm over the domain
l2norm = sqrt(sum(E.^2) * dx * dy)

# Relative L2 error (normalized by true‐solution norm)
u2norm = sqrt(sum(u_real.^2) * dx * dy)
rel_L2 = l2norm / u2norm

println("RMSE = ", rmse)
println("L2 norm = ", l2norm)
println("Relative L2 = ", rel_L2)

RMSE = 0.002146716505067564
L2 norm = 0.01718446562306548
Relative L2 = 0.013711219806291846
